In [1]:
import pandas as pd
import numpy as np
import h5py
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [2]:
data_file = "../processed_data/Self_nonSelf/pre_tokenized_peptides_dataset.hdf5"

In [3]:
with h5py.File(data_file, "r") as f:
    print(f.keys())
    print(f['reference_human_peptides'])
    print(f['nonself_peptides'])
    self_peptides = f['reference_human_peptides'][:20]
    nonself_peptides = f['nonself_peptides'][:100]    

<KeysViewHDF5 ['nonself_peptides', 'reference_human_peptides']>
<HDF5 dataset "reference_human_peptides": shape (52208587, 12), type "|u1">
<HDF5 dataset "nonself_peptides": shape (790551832, 12), type "|u1">


In [4]:
np.unique(nonself_peptides, axis=0)

array([[ 0,  2, 16, ..., 17, 14, 22],
       [ 0,  8, 15, ..., 12,  0,  9],
       [ 0, 12, 12, ..., 22, 22, 22],
       ...,
       [20,  3, 13, ..., 22, 22, 22],
       [20, 15,  4, ...,  7,  9, 22],
       [20, 20,  3, ..., 14, 22, 22]], dtype=uint8)

In [7]:
nonself_peptides[0]

TypeError: sequence item 0: expected str instance, numpy.uint8 found

In [ ]:
ixs = [2, 10, 11, 5]
mat[ixs]

In [ ]:
ixs = np.array(ixs)

In [ ]:
mat[ixs]

In [ ]:
val_size = 10000
# train_pos, val_pos = train_test_split(

In [ ]:
np.random.shuffle(nonself_peptides)

In [ ]:
train_pos, val_pos = train_test_split(self_peptides, test_size=val_size)

In [ ]:
train_pos.shape

In [ ]:
val_pos.shape

In [ ]:
train_neg, val_neg = train_test_split(nonself_peptides, test_size=val_size)

In [ ]:
def split_pretokenized_data(h5py_file, holdout_sizes=[], random_state=None):
    with h5py.File(h5py_file, "r") as f:
        print("Loading data")
        self_peptides = f['reference_human_peptides'][:]
        nonself_peptides = np.unique(f['nonself_peptides'], axis=0)    
        print("Removed {} duplicates".format(len(f['nonself_peptides'])-len(nonself_peptides)))
    print(self_peptides.shape)
    if isinstance(holdout_sizes, (int, float)):
        holdout_sizes = [holdout_sizes]
        
    print("Shuffling data")
    np.random.seed(random_state)
    np.random.shuffle(self_peptides)
    np.random.shuffle(nonself_peptides)
    
    print("Splitting data")
    peptides_sets = []
    for s in tqdm(holdout_sizes):
        holdout_pos, self_peptides = np.split(self_peptides, [s//2])
        holdout_neg, nonself_peptides = np.split(nonself_peptides, [s//2])      
        peptides_sets.append((holdout_pos, holdout_neg))

    peptides_sets.append((self_peptides, nonself_peptides))
    
    return peptides_sets
        

In [ ]:
results = split_pretokenized_data(data_file, holdout_sizes=[5000, 10000], random_state=42)

In [ ]:
val_set, ref_set, train_set = results

In [ ]:
val_set

In [ ]:
train_set[0].shape

In [ ]:
train_set[1].shape

In [ ]:
class PreSplit_Self_NonSelf_PeptideDataset(Dataset):
    def __init__(self, pos_class_data, neg_class_data,
                 negative_label=-1):
        self.pos_class_data = pos_class_data
        self.neg_class_data = neg_class_data
        self.negative_label = negative_label        
        
        self.n_negative_samples = len(neg_class_data)
        self.n_positive_samples = len(pos_class_data)
        self.labels = [negative_label, 1]

    def __len__(self):
        return len(self.pos_class_data) + len(self.neg_class_data)
    
    def __getitem__(self, idx):
        mod_idx = idx//2
        sample_class = idx % 2
        if sample_class==0:
            mod_idx = mod_idx % self.n_negative_samples
            peptide = self.neg_class_data[mod_idx]
        elif sample_class==1:
            mod_idx = mod_idx % self.n_positive_samples
            peptide = self.pos_class_data[mod_idx]
        return peptide, self.labels[sample_class]
    

In [ ]:
dset = PreSplit_Self_NonSelf_PeptideDataset(*train_set)

In [ ]:
dset[2]

In [ ]:
dloader = DataLoader(dset, batch_size=4)

batch = next(iter(dloader))

In [ ]:
batch

In [ ]:
400000*32*4